# CPU + GPU temperature comparison 
# Raspberry PI, Orange PI, Rockpi
## Benchmark: Solve N-Queens problem, single threaded and multi threaded


In [29]:
from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook, output_file
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool, ZoomInTool
from bokeh.layouts import gridplot
import pandas

In [30]:
data_orange_pi_st = pandas.read_csv("sbc_performance_data/orange_pi/zero_lts/buster/singlethread_output.csv")
data_raspberry_pi_st = pandas.read_csv("sbc_performance_data/raspberry_pi/buster/4B/std_kernel_4B_singlethread_output.csv")
data_rockpi_st = pandas.read_csv("sbc_performance_data/rockpi_s/stretch_4.4.143/singlethread_output.csv")

#remove last 5 rows of rockpi dataset to make consistent with remaining datasets 
data_rockpi_st = data_rockpi_st.loc[0:44]

device_dfs = [data_orange_pi_st,data_raspberry_pi_st,data_rockpi_st]

#add index column
[dfs.reset_index(inplace=True) for dfs in device_dfs]
#Start time at 0 
for dev in device_dfs:
    dev['time'] -= dev['time'][0]

#CPU, gpu + device data
data_orange_pi_st['CPU'] = '4x Cortex-A7'
data_raspberry_pi_st['CPU'] = '4x Cortex-A72'
data_rockpi_st['CPU'] = '2x Cortex-A72 + 4x Cortex-A53'

data_orange_pi_st['GPU'] = 'Mali400MP2 GPU @600MHz'
data_raspberry_pi_st['GPU'] = 'Broadcom VideoCore VI'
data_rockpi_st['GPU'] = 'ARM Mali-T860MP4' 

data_orange_pi_st['device'] = 'orange_pi'
data_raspberry_pi_st['device'] = 'raspberry_pi'
data_rockpi_st['device'] = 'rockpi'

data_orange_pi_st.tail()

,index,time,queen,threads,solutions,seconds,microseconds,cpu_temp,gpu_temp,CPU,GPU,device
40,40,10068,12,1,14200,252,772202,29.77,29.77,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi
41,41,10321,12,1,14200,252,902859,30.49,30.49,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi
42,42,10573,12,1,14200,251,723159,30.37,30.37,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi
43,43,10825,12,1,14200,251,498046,30.86,30.25,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi
44,44,11076,12,1,14200,251,659394,29.77,29.77,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi


In [31]:
for dev in device_dfs:
    dev['time'] -= dev['time'][0]
    
#Duration
data_orange_pi_st_timestamp = data_orange_pi_st.time/60
data_raspberry_pi_st_timestamp = data_raspberry_pi_st.time/60
data_rockpi_st_timestamp = data_rockpi_st.time/60

#Time to sole n queens
data_orange_pi_st_time = data_orange_pi_st.seconds + data_orange_pi_st.microseconds/1000000
data_raspberry_pi_st_time = data_raspberry_pi_st.seconds + data_raspberry_pi_st.microseconds/1000000
data_rockpi_st_time = data_rockpi_st.seconds + data_rockpi_st.microseconds/1000000

#add columns to dataframe
data_orange_pi_st['duration'] = data_orange_pi_st_time
data_orange_pi_st['time_to_solve'] = data_orange_pi_st_timestamp
data_raspberry_pi_st['duration'] = data_raspberry_pi_st_time
data_raspberry_pi_st['time_to_solve'] = data_raspberry_pi_st_timestamp
data_rockpi_st['duration'] = data_rockpi_st_time
data_rockpi_st['time_to_solve'] = data_rockpi_st_timestamp

#Create Bokeh data sources 
orange_ds = ColumnDataSource(data_orange_pi_st)
orange_ds.name =  "orange_ds"
raspberry_pi_ds = ColumnDataSource(data_raspberry_pi_st)
raspberry_pi_ds.name =  "raspberry_pi_ds"
rockpi_ds = ColumnDataSource(data_rockpi_st)
rockpi_ds.name =  "rockpi_ds"

In [32]:
#Bokeh plot
f = figure()

#title
f.title.text="12 Queens - Singlethreaded SBC perf"
f.title.align="center"

#instantiate and confiture hover tool 
htool = HoverTool()
ztool = ZoomInTool()
htool.tooltips = [
    ("cpu_temp", "@cpu_temp"),
    ("gpu_temp", "@gpu_temp"),
    ("CPU","@CPU"),
    ("GPU","@GPU"),
    ("Time to solve","@duration")
]
f.add_tools(htool, ztool)

#Configure style like - returns a dict, use with **style_line(args) to unpack into line function 
def style_line(x,y,source_in, width=2):
    name = source_in.name
    colormap = {"orange_ds": "Orange", "rockpi_ds": "Grey", "raspberry_pi_ds": "Crimson"}
    color = colormap.get(name)
    dash = 'solid'
    if y == "cpu_temp":
        l_label = f"{name}: cpu temp"
    elif y == "gpu_temp":
        l_label = f"{name}: gpu temp"
        dash = 'dashed'
    elif y == 'duration':
        l_label = f"{name} time to solve"
        dash = 'dotted'

    return {'x':x,'y':y,'line_width':width, 'line_dash': dash, 'line_color':color, 'source':source_in,'legend_label':l_label}

#create lines for selected data points in sources 
sources = [orange_ds,raspberry_pi_ds,rockpi_ds]

for source in sources:
    for data in ['cpu_temp','gpu_temp','duration']:
        #unpack dict from style_line function 
        f.line(**style_line('index',data,source))

#legend styling
f.legend.location='top_left'
f.legend.click_policy='hide'
f.legend.background_fill_alpha = 0.2
f.legend.border_line_color=None
f.legend.margin = 10
f.legend.padding = 18
f.legend.level = 'annotation'

#axes styling
f.xaxis.axis_label = "time (min)"
f.yaxis.axis_label = "temp (deg c) / time to solve (sec)"
f.axis.axis_label_text_color="grey"
f.xaxis.minor_tick_line_color="grey"
f.yaxis.major_label_orientation="horizontal"
f.xaxis.visible=True

#TODO: implement filtering of indices

In [33]:
data_orange_pi_mt = pandas.read_csv("sbc_performance_data/orange_pi/zero_lts/buster/multithread_output.csv")
data_raspberry_pi_mt = pandas.read_csv("sbc_performance_data/raspberry_pi/buster/4B/std_kernel_4B_multithread_output.csv")
data_rockpi_mt = pandas.read_csv("sbc_performance_data/rockpi_s/stretch_4.4.143/multithread_output.csv")

#Select rows to make consistent with single thread
data_orange_pi_mt = data_orange_pi_mt.loc[0:44]
data_raspberry_pi_mt = data_raspberry_pi_mt[0:44]
data_rockpi_mt = data_rockpi_mt[0:44]

device_dfs_mt = [data_orange_pi_mt,data_raspberry_pi_mt,data_rockpi_mt]

[dfs.reset_index(inplace=True) for dfs in device_dfs]

for dev in device_dfs_mt:
    dev['time'] -= dev['time'][0]

data_orange_pi_mt['CPU'] = '4x Cortex-A7'
data_raspberry_pi_mt['CPU'] = '4x Cortex-A72'
data_rockpi_mt['CPU'] = '2x Cortex-A72 + 4x Cortex-A53'

data_orange_pi_mt['GPU'] = 'Mali400MP2 GPU @600MHz'
data_raspberry_pi_mt['GPU'] = 'Broadcom VideoCore VI'
data_rockpi_mt['GPU'] = 'ARM Mali-T860MP4' 

data_orange_pi_mt['device'] = 'orange_pi'
data_raspberry_pi_mt['device'] = 'raspberry_pi'
data_rockpi_mt['device'] = 'rockpi'

#Time elapsed
data_raspberry_pi_mt_timestamp = data_raspberry_pi_mt.time/60
data_rockpi_mt_timestamp = data_rockpi_mt.time/60
data_orange_pi_mt_timestamp = data_orange_pi_mt.time/60

#Time for device to solve queens
data_orange_pi_mt_time = data_orange_pi_mt.seconds + data_orange_pi_mt.microseconds/1000000
data_raspberry_pi_mt_time = data_raspberry_pi_mt.seconds + data_raspberry_pi_mt.microseconds/1000000
data_rockpi_mt_time = data_rockpi_mt.seconds + data_rockpi_mt.microseconds/1000000

#add columns to dataframe
data_raspberry_pi_mt['duration'] = data_raspberry_pi_mt_time
data_raspberry_pi_mt['time_to_solve'] = data_raspberry_pi_mt_timestamp
data_orange_pi_mt['duration'] = data_orange_pi_mt_time
data_orange_pi_mt['time_to_solve'] = data_orange_pi_mt_timestamp
data_rockpi_mt['duration'] = data_rockpi_mt_time
data_rockpi_mt['time_to_solve'] = data_rockpi_mt_timestamp

#Create bokeh specific data sources
orange_mt_ds = ColumnDataSource(data_orange_pi_mt) 
orange_mt_ds.name =  "orange_ds"
raspberry_pi_mt_ds = ColumnDataSource(data_raspberry_pi_mt)
raspberry_pi_mt_ds.name =  "raspberry_pi_ds"
rockpi_mt_ds = ColumnDataSource(data_rockpi_mt)
rockpi_mt_ds.name =  "rockpi_ds"
data_orange_pi_mt.tail()

,time,queen,threads,solutions,seconds,microseconds,cpu_temp,gpu_temp,CPU,GPU,device,duration,time_to_solve
40,2763,12,4,14200,70,66996,68.49,68.49,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi,70.066996,46.050000
41,2833,12,4,14200,69,995526,70.30,68.36,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi,69.995526,47.216667
42,2904,12,4,14200,70,291187,68.24,68.24,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi,70.291187,48.400000
43,2975,12,4,14200,71,526593,68.00,67.16,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi,71.526593,49.583333
44,3045,12,4,14200,70,92285,69.70,69.70,4x Cortex-A7,Mali400MP2 GPU @600MHz,orange_pi,70.092285,50.750000


In [34]:
g = figure()

g.add_tools(htool)

#create lines for selected data points in sources 
sources_mt = [orange_mt_ds,raspberry_pi_mt_ds,rockpi_mt_ds]

for source in sources_mt:
    for data in ['cpu_temp','gpu_temp','duration']:
        #unpack dict from style_line function 
        g.line(**style_line('index',data,source))

#legend styling
g.legend.location='top_left'
g.legend.click_policy='hide'
g.legend.background_fill_alpha = 0.2
g.legend.border_line_color=None
g.legend.margin = 10
g.legend.padding = 18
g.legend.level = 'annotation'

#axes styling
g.xaxis.axis_label = "time (min)"
g.yaxis.axis_label = "temp (deg c) / time to solve n queens (sec)"
g.axis.axis_label_text_color="grey"
g.xaxis.minor_tick_line_color="grey"
g.yaxis.major_label_orientation="horizontal"
g.xaxis.visible=True

#titles
g.title.text="12 Queens - Multihreaded SBC perf"
g.title.align="center"

In [35]:
#show in html
output_file("sbc_data_exploration.html", "Single Board Computer Data Viz")
show(gridplot([[f,g]]))
